Manipolazione dati e popolamento delle tabelle Utente, Birra, Consumazione

In [34]:
# Funzioni e imports
import json, os, random,datetime
import mysql.connector

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")


def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

def randomDate(datareg):
    end_date = datetime.datetime.now()
    random_date = (datareg + (end_date - datareg) * random.random()).strftime('%Y-%m-%d %H:%M:%S')
    return(random_date)


In [2]:
# Connessione al DB
db = mysql.connector.connect(host="localhost", user="arzo", password="abcd1234", database="beer")
cursor = db.cursor()

In [ ]:
jsonUtenti = read_json("utente.json")

#Dati casuali generati con mockaroo.com, ma l'attributo 'conto' era in forma "123,45", mentre il DB necessita di 123.45, quindi converto
for user in jsonUtenti:
    user["saldo"] = float(user["saldo"].replace(",", "."))
    print(user["saldo"])

# Magari serve più tardi...
serialize_json("./", "utente.json", jsonUtenti) 

# Popolamento Utente
for utente in jsonUtenti:
    cursor.execute(
        f'INSERT INTO Utente(id,nome, cognome, email, psw, saldo, data_reg) VALUES (\'{utente["id"]}\',\'{utente["nome"]}\',\'{utente["cognome"]}\',\'{utente["email"]}\',\'{utente["psw"]}\',{utente["saldo"]},\'{utente["data_reg"]}\')'
    )
    
db.commit()

In [ ]:
jsonBirre = read_json("birra.json")

#Alcune birre 
for birra in jsonBirre:
    cursor.execute(f'INSERT INTO Birra(id, nome, prezzo_litro, disp, gradi, tipo) VALUES ({birra["id"]},\'{birra["nome"]}\',{birra["prezzo_litro"]},\'{birra["disp"]}\',{birra["gradi"]},\'{birra["tipo"]}\');')
    #print(f'Inserita birra {birra["nome"]}')

db.commit()

In [ ]:
cursor.execute('SELECT id FROM Utente;')
utenti = cursor.fetchall()
cursor.execute('SELECT data_reg FROM Utente;')
datareg = cursor.fetchall()




In [38]:
cursor.execute("DELETE FROM Consumazione")
db.commit()

cursor.execute('SELECT id FROM Utente;')
utenti = cursor.fetchall()

cursor.execute('SELECT data_reg FROM Utente;')
datareg = cursor.fetchall()

cursor.execute('SELECT id FROM Birra WHERE disp=\'DISP\';')
id_birre = cursor.fetchall()

for i in range(200):
    #Scelgo un numero casuale tra 0 e dim. utenti
    rnd=random.randint(0,len(utenti)-1) 

    cursor.execute(f'INSERT INTO Consumazione(id,user_id,beer_id,quantita,data_consumazione) VALUES ({random.randint(1,10000000)},\'{utenti[rnd][0]}\',{random.choice(id_birre)[0]},{random.choice([125,500,750,1000])},\'{randomDate(datareg[rnd][0])}\');')

db.commit()